In [5]:
import os
import json
import requests
from dotenv import load_dotenv

# 認証情報
auth_url = os.getenv('AICORE_AUTH_URL').strip(' "\',')
client_id = os.getenv('AICORE_CLIENT_ID').strip(' "\',')
client_secret = os.getenv('AICORE_CLIENT_SECRET').strip(' "\',')
base_url = os.getenv('AICORE_BASE_URL').strip(' "\',')
resource_group = os.getenv('AICORE_RESOURCE_GROUP').strip(' "\',')

# トークンを取得する関数
def get_token(auth_url, client_id, client_secret):
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }
    response = requests.post(auth_url + '/oauth/token', data=data)
    response.raise_for_status()
    return response.json()['access_token']

# トークンを取得
token = get_token(auth_url, client_id, client_secret)

# API にアクセスする例
headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json',
    'AI-Resource-Group': resource_group
}

# /v2/lm/scenarios エンドポイントに対してリクエストを送信
response = requests.get(base_url + '/v2/lm/scenarios', headers=headers)

# レスポンスのステータスコードと内容を表示
print(f'Status Code: {response.status_code}')
print(f'Response Text: {response.text}')

Status Code: 200
Response Text: {
  "count": 5,
  "resources": [
    {
      "createdAt": "2023-11-21T09:41:52+00:00",
      "description": "AI Core Global Scenario for LLM Access",
      "id": "foundation-models",
      "labels": [
        {
          "key": "scenarios.ai.sap.com/llm",
          "value": "true"
        }
      ],
      "modifiedAt": "2024-04-08T07:44:29+00:00",
      "name": "foundation-models"
    },
    {
      "createdAt": "2024-03-26T05:11:20+00:00",
      "description": "yynhr Introduction to SAP AI Core",
      "id": "yynhr-learning",
      "modifiedAt": "2024-03-26T05:11:20+00:00",
      "name": "yynhr-Tutorial"
    },
    {
      "createdAt": "2024-03-26T10:01:19+00:00",
      "description": "yynhr-Tutorial to add custom code to SAP AI Core",
      "id": "yynhr-learning-code",
      "modifiedAt": "2024-03-26T10:01:19+00:00",
      "name": "yynhr-Code (Tutorial)"
    },
    {
      "createdAt": "2024-04-20T13:42:22+00:00",
      "description": "Tutorial to add 

In [11]:
# 設定の作成リクエストデータ
configuration_data = {
    "name": "gpt-35-itomond",
    "executableId": "azure-openai",
    "scenarioId": "foundation-models",
    "versionId": "0.0.1",
    "parameterBindings": [
        {"key": "modelName", "value": "gpt-35-turbo-0125"},
        {"key": "modelVersion", "value": "latest"}
    ],
    "inputArtifactBindings": []
}

# 設定の作成リクエスト
response_config = requests.post(base_url + '/v2/lm/configurations', headers=headers, data=json.dumps(configuration_data))

# エラーの詳細を表示
try:
    response_config.raise_for_status()
except requests.exceptions.HTTPError as err:
    print(f'HTTP error occurred: {err}')  # HTTP エラーメッセージ
    print(f'Response content: {response_config.content}')  # レスポンスの内容

# レスポンスの表示
configuration = response_config.json()
print(configuration)

{'id': 'ded2d0e0-12bc-4e58-b0e2-2563e8c68cad', 'message': 'Configuration created'}


In [12]:
import time

# デプロイメントの作成リクエストデータ
deployment_data = {
    "configurationId": configuration['id']
}

# デプロイメントの作成リクエスト
response_deployment = requests.post(base_url + '/v2/lm/deployments', headers=headers, data=json.dumps(deployment_data))

# エラーの詳細を表示
try:
    response_deployment.raise_for_status()
except requests.exceptions.HTTPError as err:
    print(f'HTTP error occurred: {err}')  # HTTP エラーメッセージ
    print(f'Response content: {response_deployment.content}')  # レスポンスの内容

# レスポンスの表示
deployment = response_deployment.json()
print(deployment)


{'deploymentUrl': '', 'id': 'd5da951d07039dd4', 'message': 'Deployment scheduled.', 'status': 'UNKNOWN'}


In [14]:
# デプロイメントの詳細取得リクエスト
deployment_id = deployment['id']  # 取得したデプロイメントIDを使用

# デプロイメントのステータスが更新されるのを待つための関数
def check_deployment_status(deployment_id):
    status = 'PENDING'
    while status == 'PENDING' or status == 'UNKNOWN':
        response_deployment_status = requests.get(base_url + f'/v2/lm/deployments/{deployment_id}', headers=headers)

        # エラーの詳細を表示
        try:
            response_deployment_status.raise_for_status()
        except requests.exceptions.HTTPError as err:
            print(f'HTTP error occurred: {err}')  # HTTP エラーメッセージ
            print(f'Response content: {response_deployment_status.content}')  # レスポンスの内容
            return None

        deployment_status = response_deployment_status.json()
        status = deployment_status.get('status', 'PENDING')

        print(f"Deployment Status: {status}")

        # ステータスが更新されるまで待機
        time.sleep(10)

    return deployment_status

# デプロイメントのステータスをチェック
deployment_status = check_deployment_status(deployment_id)
print(deployment_status)

Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: UNKNOWN
Deployment Status: PENDING
Deployment Status: PENDING
Deployment Status: PENDING
Deployment Status: PENDING
Deployment Status: RUNNING
{'configurationId': 'ded2d0e0-12bc-4e58-b0e2-2563e8c68cad', 'configurationName': 'gpt-35-itomond', 'createdAt': '2024-05-26T04:46:44Z', 'deploymentUrl': 'https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d5da951d07039dd4', 'details': {'resources': {'backend_details': {'model': {'name': 'gpt-35-turbo-0125', 'version': 'latest'}}}, 'scaling': {'backend_details': {}}}, 'id'